In [30]:
pip install openpyxl

In [31]:
## Load modules and read the 'Example_Data' sheet ##
import numpy as np
import pandas as pd

example_data = pd.read_excel('/kaggle/input/example-data/Example_Data.xlsx',sheet_name=0,skiprows=1,header=0)
print(example_data.head())
print(example_data.info())

In [32]:
## Clean up duplicated company id - company name pairs ##
for id in example_data['Company ID'].unique():
    if example_data[example_data['Company ID']==id]['Company Name'].nunique() > 1:
        example_data = example_data[example_data['Company ID'] != id]
for name in example_data['Company Name'].unique():
    if example_data[example_data['Company Name']==id]['Company ID'].nunique() > 1:
        example_data = example_data[example_data['Company Name'] != name]
print(example_data.info())

In [33]:
## Inspect the 'Fiscal Year' column and drop invalid values
print(example_data['Fiscal Year'].unique())
example_data = example_data[example_data['Fiscal Year'].isin([213,'2Q16']) == False]
print(example_data['Fiscal Year'].unique())
example_data['Fiscal Year'] = example_data['Fiscal Year'].astype(int)

In [34]:
## Inspect the 'SIC Code' column and drop invalid values ##
print(example_data['SIC Code'].unique())
example_data = example_data[example_data.apply(lambda x: type(x['SIC Code']) == int and len(str(x['SIC Code'])) == 4,axis=1)]
print(example_data['SIC Code'].unique())
example_data['SIC Code'] = example_data['SIC Code'].astype(int)

In [35]:
## Clean up the 'Trading Currency' column ##
print(example_data['Trading Currency'].unique())
example_data = example_data[example_data['Trading Currency'].isin(['USD','GBP'])]
print(example_data['Trading Currency'].unique())

In [36]:
## Clean up the 'CDS' column by filling NAs with -999 ##
example_data['CDS'] = example_data['CDS'].fillna(-999)
example_data['CDS'] = example_data['CDS'].astype(int)

In [37]:
## Melt data ##
example_data = example_data.melt(id_vars=['Company ID','Company Name','Fiscal Year','Industry','SIC Code','Trading Currency'],value_vars=['SP','CDS','APD','ARD','ADA'],var_name='Metric Name',value_name='Value')
print(example_data.info())

In [38]:
## Read the 'Example_DB' sheet ##
example_db = pd.read_excel('/kaggle/input/example-data/Example_Data.xlsx',sheet_name=1,header=0)
example_db['Value'] = example_db['Value'].fillna(-999)
example_db['Value'] = example_db['Value'].astype(int)
print(example_db.head())
print(example_db.info())

In [39]:
## Merge two dataframes and find the different data entries ##
data_merge = example_db.merge(example_data, how='outer', on=['Company ID','Company Name','Fiscal Year','Industry','SIC Code','Trading Currency','Metric Name'], suffixes=['_data','_db'])
diffs = data_merge[data_merge.apply(lambda x: x['Value_data'] == np.nan or x['Value_db'] == np.nan or x['Value_data']!=x['Value_db'], axis=1)]
diffs['ERROR Type'] = 'Unequal'
diffs.loc[diffs['Value_data'].isna(),'ERROR Type'] = 'Not_in_File'
diffs.loc[diffs['Value_db'].isna(),'ERROR Type'] = 'Not_in_DB'
diffs = diffs.replace(-999,np.nan)

In [41]:
## Print the result ##
diffs = diffs.rename(columns = {'Value_data':'Data in File','Value_db':'Data in DB'})
print(diffs)